In [1]:
import yelp_functions
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import ast
import datetime
from dateutil.rrule import rrule, MONTHLY
import matplotlib.pyplot as plt
from collections import Counter, OrderedDict
from IPython.display import clear_output, display

In [2]:
tips = pd.read_json('yelp_academic_dataset_tip.json', lines=True)
#assign datetime
tips_datetimed = tips.assign(datetime = pd.to_datetime(tips['date'])).drop(labels=['date'], axis = 'columns').set_index('datetime').sort_index()



In [3]:
def transform_tips_data(df,weight):
    dictionary ={}
    gbym = df.groupby([df.index.year.rename('year'),df.index.month.rename('month')])
    
    for key,records in gbym: 
        gb_df = gbym.get_group(key)
        bids = gb_df['business_id'].unique()
        for i in bids: 
            bid_records = gb_df.loc[gb_df['business_id'] == i]
            num_tips_weighted = len(bid_records)+ sum(bid_records['likes'].values)*weight
            dictionary[str(key)+'%BREAK%'+i] = num_tips_weighted
    
    month_date = [d.split('%BREAK%')[0] for d in list(dictionary.keys())]
    bids = [b.split('%BREAK%')[1] for b in list(dictionary.keys())]
    num_weighted_tips = [s for s in dictionary.values()] 
    
    groupbydate =  [str((x.year,x.month)) for x in df.index]
    df['groupbydate'] = groupbydate
    
    tips_transform = pd.DataFrame({"business_id":bids,
                    "num_weighted_tips": num_weighted_tips,"month_date" : month_date})
    
    df = df.reset_index().merge(tips_transform, how="left",left_on = ['business_id','groupbydate'], right_on = ['business_id','month_date']).set_index('datetime')
    df.drop(['likes', 'text','groupbydate','month_date'], axis=1,inplace = True)
    
    return df

            

In [4]:
check = transform_tips_data(tips_datetimed[200000:220000],.02)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
testgb = check.groupby([check.index.year.rename('year'),check.index.month.rename('month'),check['business_id']])

In [6]:
for key,record in testgb:
    if len(testgb.get_group(key)) > 5:
        checking_tips_sum_per_bid = testgb.get_group(key)
    

In [7]:
checking_tips_sum_per_bid

,business_id,user_id,num_weighted_tips
datetime,,,
2012-03-02,zcScEL0WEdFkROcnz5379g,qgEVPFVPcXZ9xeuBgZdxdQ,8.0
2012-03-02,zcScEL0WEdFkROcnz5379g,XXdKvXR9GGWzGhz3Z075Zg,8.0
2012-03-02,zcScEL0WEdFkROcnz5379g,OXwlm-w6TBs6h33A7UzvsA,8.0
2012-03-02,zcScEL0WEdFkROcnz5379g,J3731fkiEXySc_p0R0-83w,8.0
2012-03-03,zcScEL0WEdFkROcnz5379g,6UhK6P81ahxBkpmbfoYt2A,8.0
2012-03-03,zcScEL0WEdFkROcnz5379g,E5xx_Z_kMAlzbIS72zY8Hw,8.0
2012-03-03,zcScEL0WEdFkROcnz5379g,zmC3gPQr0KPmsL3BinasJg,8.0
2012-03-03,zcScEL0WEdFkROcnz5379g,4yG4J05aKzE2zov0Jr37kg,8.0


In [8]:
tips_datetimed.loc[(tips_datetimed['business_id'] == 'zcScEL0WEdFkROcnz5379g') 
                   & ((tips_datetimed.index == pd.to_datetime('2012-03-02')) | (tips_datetimed.index == pd.to_datetime('2012-03-03')))]
        

,business_id,likes,text,user_id
datetime,,,,
2012-03-02,zcScEL0WEdFkROcnz5379g,0,Dinner w the bros and pops in Vegas...let's go..!,qgEVPFVPcXZ9xeuBgZdxdQ
2012-03-02,zcScEL0WEdFkROcnz5379g,0,Balling on a budget.,XXdKvXR9GGWzGhz3Z075Zg
2012-03-02,zcScEL0WEdFkROcnz5379g,0,The whole shabang!,OXwlm-w6TBs6h33A7UzvsA
2012-03-02,zcScEL0WEdFkROcnz5379g,0,The Whole Shebang is the way to go!,J3731fkiEXySc_p0R0-83w
2012-03-03,zcScEL0WEdFkROcnz5379g,0,I like hot n juicy better,6UhK6P81ahxBkpmbfoYt2A
2012-03-03,zcScEL0WEdFkROcnz5379g,0,Love the Snow Crab!,E5xx_Z_kMAlzbIS72zY8Hw
2012-03-03,zcScEL0WEdFkROcnz5379g,0,First time. Bigger than Hot-n-Juicy.,zmC3gPQr0KPmsL3BinasJg
2012-03-03,zcScEL0WEdFkROcnz5379g,0,Life just doesn't get any better than this.,4yG4J05aKzE2zov0Jr37kg
